In [1]:
modello_per_analisi = 'Audi_A3'
# Opzioni possibili: Mercedes_Classe_A, BMW_Serie_1, Audi_A3, Fiat_Panda

### 0. Import funzioni e librerie

In [2]:
import pandas as pd
import os
import sys
import requests
import importlib
from pathlib import Path


In [3]:
project_root = Path.cwd().parents[1]

# Percorsi delle cartelle contenenti i moduli
loader_dir = project_root / 'Source' / '2_Data_Preparation' / 'Utils'
utils_dir = project_root / 'Source' / '3_Data_Analysis' / 'Utils'
# Aggiungi al sys.path
sys.path.extend([str(loader_dir), str(utils_dir)])

# Import
from data_loader import load_csv_for_notebook
from Plots_maker import plot_is_conveniente_distribution, plot_price_prediction_distributions
from Function_price_analysis import modello_ml, predizione_prezzo, is_conveniente_class_report, indice_appetibilita, display_top_auto

In [4]:
# --- IMPORT CONFIGURAZIONE DELL'AUTO ---
#Configurazione path per import dinamico del modulo di configurazione 

config_path = project_root / 'Data'/ 'config'

sys.path.extend([str(config_path)])
print("\n - config_path:", config_path)
print(" - Contenuto della cartella:", os.listdir(config_path))

# --- Import dinamico del modulo di configurazione in base al modello ---
modulo_config = f"config_{modello_per_analisi}"  # esempio: config_Mercedes_Classe_A
try:
    config = importlib.import_module(modulo_config)
except ModuleNotFoundError:
    print(f"❌ Modulo di configurazione '{modulo_config}' non trovato.")
    sys.exit(1)
# --- Accesso ai parametri di configurazione ---
#allestimenti = config.allestimenti
allestimento_performance = config.allestimento_performance
allestimento_sport = config.allestimento_sport
allestimento_middle = config.allestimento_middle
allestimento_base = config.allestimento_base


 - config_path: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/config
 - Contenuto della cartella: ['config_Audi_A3.py', 'config_Opel_Corsa.py', '__pycache__', 'config_BMW_Serie_1.py', 'config_Mercedes_Classe_A.py']


### 1. Dati

In [5]:
#### CARICAMENTO DATI SINGOLI
data_dummy= load_csv_for_notebook(project_root,['Data', 'Processed_data', modello_per_analisi, f'data_dummy_{modello_per_analisi}.csv'])
data = load_csv_for_notebook(project_root,['Data', 'Processed_data', modello_per_analisi, f'data_{modello_per_analisi}.csv'])

X = data_dummy.drop(columns=['Prezzo', 'Distanza', 'Venditore'])
y = data_dummy['Prezzo']

✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/Audi_A3/data_dummy_Audi_A3.csv (3609 righe, 15 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/Audi_A3/data_Audi_A3.csv (3609 righe, 18 colonne)


### 2. Predizione prezzo, analisi della convenienza

In [6]:
modello_ml(X, y, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))
data_ml, y_pred = predizione_prezzo(data_dummy, X, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))

Valutazione delle performance sul test set
 - Mean Squared Error: 14658274.92
 - Root Mean Squared Error: 3828.61 euro
 - R-squared:  0.86


In [7]:
plot_is_conveniente_distribution(data_ml)


Distribuzione relativa della variabile 'is_conveniente':

 is_conveniente  Frequenza Relativa
              0            0.517318
              1            0.482682


In [8]:
plot_price_prediction_distributions(data_ml)

In [9]:
#is_conveniente_class_report(data_ml)

### 3. Indice di attendibilità

In [13]:
#Prezzo soglia: prezzo oltre il quale il punteggio di attendibilità diminuisce
prezzo_soglia=35000

#Valori per il calcolo indice di attendibilità
valore_idx_anni = 0.025
valore_idx_prezzo = 0.15
valore_idx_km= 0.15 
valore_idx_dist = 0.10 
valore_idx_allestimento= 0.25 
valore_idx_cv =0.025
valore_idx_cambio = 0.20 

In [14]:
df = indice_appetibilita(data, y_pred, prezzo_soglia, valore_idx_anni, valore_idx_prezzo, valore_idx_km, 
                           valore_idx_dist, valore_idx_allestimento, valore_idx_cv, valore_idx_cambio, 
                           allestimento_performance, allestimento_sport,allestimento_middle,allestimento_base)

In [16]:
prezzo_max = 32000
prezzo_min = 24000
km_max =40000
km_min = 0
dist_max = 150
dispaly = 20

display_top_auto(df, dispaly, prezzo_max, prezzo_min, km_max, km_min, dist_max, carburante=None)


Le migliori 20 auto secondo l'indice di appetibilità sono:


,Annuncio,Link,Prezzo,Anni,Immatricolazione,Chilometraggio,Cambio,Carburante,CV,Allestimento,...,Venditore,Distanza,Comune,CAP,Regione,Area,Indice_Appetibilità,prezzo_previsto,delta_prezzo,is_conveniente
1983,AUDI A3 SPB 40 TFSI e S tronic,https://www.automobile.it/pieve-cento-audi-a3-...,26950,3,2022,26,automatico,Ibrido,190,S,...,Rivenditore,14,pieve cento,40066,Emilia-Romagna,Nord-est,0.85,27803.58,853.58,1
1990,AUDI A3 SPB 40 TFSI e S tronic S line edition ...,https://www.automobile.it/parma-audi-a3-spb-40...,27000,3,2022,72,automatico,Ibrido,190,S line,...,Rivenditore,65,parma,43121,Emilia-Romagna,Nord-est,0.85,28942.95,1942.95,1
2017,AUDI A3 Sedan 30 TDI S line edition,https://www.automobile.it/castenaso-audi-a3-se...,27900,2,2023,26,automatico,Diesel,116,S line,...,Rivenditore,31,castenaso,40055,Emilia-Romagna,Nord-est,0.85,29271.16,1371.16,1
2088,AUDI A3 SPB 30 TDI S tronic,https://www.automobile.it/bologna-audi-a3-spb-...,29099,2,2023,21,automatico,Diesel,116,S,...,Rivenditore,25,bologna,40121,Emilia-Romagna,Nord-est,0.85,29099.00,0.00,0
2117,AUDI A3 SPB 30 TDI S tronic S line edition,https://www.automobile.it/castenaso-audi-a3-sp...,29900,1,2024,35,automatico,Diesel,116,S line,...,Rivenditore,31,castenaso,40055,Emilia-Romagna,Nord-est,0.85,31461.27,1561.27,1
2118,AUDI A3 SPB 30 TFSI S tronic S line edition,https://www.automobile.it/castenaso-audi-a3-sp...,29900,1,2024,44,automatico,Ibrido,110,S line,...,Rivenditore,31,castenaso,40055,Emilia-Romagna,Nord-est,0.85,35441.50,5541.50,1
2142,Audi A3 SPB 40 TFSI e S tronic S LINE Plug-In ...,https://www.automobile.it/casalgrande-audi-a3-...,30499,3,2022,20,automatico,Ibrido,190,S line,...,Rivenditore,32,casalgrande,42013,Emilia-Romagna,Nord-est,0.85,31884.20,1385.20,1
2155,AUDI A3 SPB 45 TFSI e S tronic S line edition,https://www.automobile.it/firenze-audi-a3-spb-...,30599,2,2023,43,automatico,Ibrido,245,S line,...,Rivenditore,99,firenze,50121,Toscana,Centro,0.85,30844.65,245.65,1
2156,AUDI A3 SPB 45 TFSI e S tronic S line edition,https://www.automobile.it/verona-audi-a3-spb-4...,30599,2,2023,43,automatico,Ibrido,245,S line,...,Rivenditore,86,verona,37121,Veneto,Nord-est,0.85,30742.55,143.55,1
3095,"Audi A3 SPB 30 g-tron S tronic S-LINE """" BELL1...",https://autosupermarket.it/annunci/audi-a3-30-...,26690,2,2023,96,automatico,Benzina,131,S line,...,Rivenditore,65,parma,43126,Emilia-Romagna,Nord-est,0.85,29750.98,3060.98,1
